I've set Matomo up on a 64-bit Ubuntu Server virtual box. If these instructions aren't working for you and you're on a different system, I would appreciate hearing about any differences or difficulties you encounter. If you cut an issue against this repo, I will incorporate your information into these instructions.

These instructions are lightly adapted from [this website](https://www.howtoforge.com/how-to-install-matomo-web-analytics-on-ubuntu-2004/).

0. Create a **username** and **password** for Matomo to access the database. Save these for later in the setup.

1. Set up the database for Matomo's parsed logs (we can skip the earlier steps because IIAB already installed MariaDB). 

    ```
    sudo mysql
    CREATE DATABASE matomodb;
    GRANT ALL ON matomodb.* TO '<<<the username you made here>>>' IDENTIFIED BY '<<<the password you made here>>>';
    FLUSH PRIVILEGES;
    EXIT;
    ```

2. Install Matomo.

    ```
    wget https://builds.matomo.org/matomo.zip
    unzip matomo.zip
    sudo chown -R www-data:www-data <<<the username you made here>>>
    sudo mv matomo /library/www/html/.
    ```

3. You can now open an Internet browser and navigate to your Internet-in-a-Box homepage. Change the "/home/" suffix to "/matomo/". On my virtual box and home network setup, for example, I navigate to Internet-in-a-Box at http://10.0.0.242/home/, and install Matomo by going to http://10.0.0.242/matomo/. You should see the Matomo installation page, as depicted [here](https://www.howtoforge.com/how-to-install-matomo-web-analytics-on-ubuntu-2004/#access-matomo-analytics).

4. Click through the first couple screens until you get to "Database Setup". Here, enter the login and password you saved in Step 1. For the Database Name, you can just put "matomodb". You should not need to change the other options.

5. Click to the next screen and create an user name and password to log in to your Matomo analytics installation. These could be the same as the ones above if you want anyone who can access the analytics suite to make arbitrary changes to the underlying database. I provided a fake e-mail; so far this hasn't been a problem.

6. Click to the next screen. Put whatever you want for the other three fields, and your Internet-in-a-Box website address in the "Website URL" field. **Note!** You probably should not use "http**s**" as in the hint; Internet-in-a-Box uses "http".

7. Click through the next couple screens. You don't need to install the tracking code. Eventually, you should get to a page that has a green button at the bottom that congratulates you and invites you to log in to Matomo. Click on that, and then you'll get to a login screen. Log in. You should now be at a page with [this aesthetic](https://m-img.org/spai/q_lossless+ret_img+to_webp/https://static.matomo.org/wp-content/uploads/2020/06/website-graphics-2020-v5-1-e1611111459689.png).

8. Right now you should have no indication of any traffic. There's one more step, which is to run the [log analytics tool](https://matomo.org/docs/log-analytics-tool-how-to/) to collect statistics. It may be a good idea to put this on a crontab eventually.

    ```
    cd /library/www/html/matomo/misc/log-analytics/
    sudo chown -R www-data:www-data /library/www/html/matomo/tmp
    sudo python3 import_logs.py --url=<<<your matomo home page that you navigated to earlier>>> --idsite=1 --recorders=4 --enable-http-errors --enable-http-redirects --enable-static --enable-bots /var/log/nginx/access.log 
    ```

    For example, for me, I ran this for the second command:

    ```
    sudo python3 import_logs.py --url=http://10.0.0.242/matomo --idsite=1 --recorders=4 --enable-http-errors --enable-http-redirects --enable-static --enable-bots /var/log/nginx/access.log
    ```

    This last step is very easy to mess up.
        - The URL should be the URL you are using **to access Matomo**, not your Internet-in-a-Box homepage, as you might intuitively assume.
        - The "idsite" argument should be "1", not anything that you have set, unless you are doing something weird and hosting other sites on your server too. In this case, follow [these instructions](https://matomo.org/faq/general/faq_19212/) to figure out your website ID.
        - You might discover at this point that you've configured your permissions incorrectly. When I moved Matomo as above, it seems like the ownership of the Matomo directory got changed. I fixed it with the final ```chown``` command I ran, but you might need to mess around a bit more. If so, please cut me an issue and I will try to fix the instructions.
    
9. If this last command worked, you should be able to navigate back to Matomo home and see your traffic to-date!
    
10. To import the latest logs into matomo, run the run the [log analytics tool](https://matomo.org/docs/log-analytics-tool-how-to/) again. NB: It seems like Matomo updates its web interface with the latest logs at a regular interval. We have observed a delay (up to 10 minutes) at times between importing new data and seeing it reflected on the site.
